In [404]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [405]:
df = pd.read_csv('flights.csv')
df_airports = pd.read_csv('airports.csv')
df_airlines = pd.read_csv('airlines.csv')
df.head(2)

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,0,363169,5290975,2015,11,28,6,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,704061,4899614,2015,11,2,1,AA,2516,N3CVAA,ORD,DEN,1715,1743.0,28.0,40.0,1823.0,164.0,177.0,126.0,888,1929.0,11.0,1859,1940.0,41.0,0,0,NaN,13.0,0.0,28.0,0.0,0.0


In [406]:
df_airports.head(2)

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819


In [407]:
df_airlines.head(2)

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.


> We will use the `df_airports` and `df_airlines` to merge them with the `df` to creat a great One dataset.

# Lets first clean the flights dataset.

In [408]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274964 entries, 0 to 274963
Data columns (total 34 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0.2         274964 non-null  int64  
 1   Unnamed: 0           274964 non-null  int64  
 2   Unnamed: 0.1         274964 non-null  int64  
 3   YEAR                 274964 non-null  int64  
 4   MONTH                274964 non-null  int64  
 5   DAY                  274964 non-null  int64  
 6   DAY_OF_WEEK          274964 non-null  int64  
 7   AIRLINE              274964 non-null  object 
 8   FLIGHT_NUMBER        274964 non-null  int64  
 9   TAIL_NUMBER          274225 non-null  object 
 10  ORIGIN_AIRPORT       274964 non-null  object 
 11  DESTINATION_AIRPORT  274964 non-null  object 
 12  SCHEDULED_DEPARTURE  274964 non-null  int64  
 13  DEPARTURE_TIME       270719 non-null  float64
 14  DEPARTURE_DELAY      270719 non-null  float64
 15  TAXI_OUT         

> We notice that there are many columns we need to edit.
>
> 1. There are 3 Unnamed columns we need to drop.
>
>
> 2. There are many Null values here. The last 6 columns have many Null values but we need from them the:
>  * `CANCELLATION_REASON`. So, we will will try to replace the Null with readable value. 
>
>
> 3. * `SCHEDULED_DEPARTURE`, `DEPARTURE_TIME`, `SCHEDULED_TIME`, `ELAPSED_TIME`, `AIR_TIME`, `SCHEDULED_ARRIVAL`, `ARRIVAL_TIME`, `WHEELS_ON`, and `WHEELS_OFF` **All needed to be converted to string to handle the missed zeros then to DateTime.**
>
>  * Also we should convert the `DAY_OF_WEEK` column to be string with the day name.

> ### 1) Droping the Un-named columns.

In [409]:
df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

> ### 2) Cleaning the `CANCELLATION_REASON` column.

In [410]:
df.CANCELLATION_REASON.value_counts()

B    2397
A    1260
C     776
Name: CANCELLATION_REASON, dtype: int64

> Now we need to replace these values with readable reasons

In [411]:
def get_reasons(reason):
    if reason == "A":
        return "CANCELLATION_REASON"
    elif reason == "B":
        return "Weather"
    elif reason == "C":
        return "National Air System"
    else:
        return "NO-CANCELLATION"

In [412]:
df['CANCELLATION_REASON'] = df.CANCELLATION_REASON.apply(lambda reason: get_reasons(reason))
df.CANCELLATION_REASON.value_counts()

NO-CANCELLATION        270531
Weather                  2397
CANCELLATION_REASON      1260
National Air System       776
Name: CANCELLATION_REASON, dtype: int64

### 3) Operating with Null values from columns.

In [413]:
# The null values in the not canceled flights.
df[df.CANCELLED == 0].isnull().sum()

YEAR                        0
MONTH                       0
DAY                         0
DAY_OF_WEEK                 0
AIRLINE                     0
FLIGHT_NUMBER               0
TAIL_NUMBER                 0
ORIGIN_AIRPORT              0
DESTINATION_AIRPORT         0
SCHEDULED_DEPARTURE         0
DEPARTURE_TIME              0
DEPARTURE_DELAY             0
TAXI_OUT                    0
WHEELS_OFF                  0
SCHEDULED_TIME              0
ELAPSED_TIME              758
AIR_TIME                  758
DISTANCE                    0
WHEELS_ON                 154
TAXI_IN                   154
SCHEDULED_ARRIVAL           0
ARRIVAL_TIME              154
ARRIVAL_DELAY             758
DIVERTED                    0
CANCELLED                   0
CANCELLATION_REASON         0
AIR_SYSTEM_DELAY       218668
SECURITY_DELAY         218668
AIRLINE_DELAY          218668
LATE_AIRCRAFT_DELAY    218668
WEATHER_DELAY          218668
dtype: int64

In [414]:
# The null values in the canceled flights.
df[df.CANCELLED == 1].isnull().sum()

YEAR                      0
MONTH                     0
DAY                       0
DAY_OF_WEEK               0
AIRLINE                   0
FLIGHT_NUMBER             0
TAIL_NUMBER             739
ORIGIN_AIRPORT            0
DESTINATION_AIRPORT       0
SCHEDULED_DEPARTURE       0
DEPARTURE_TIME         4245
DEPARTURE_DELAY        4245
TAXI_OUT               4393
WHEELS_OFF             4393
SCHEDULED_TIME            0
ELAPSED_TIME           4433
AIR_TIME               4433
DISTANCE                  0
WHEELS_ON              4433
TAXI_IN                4433
SCHEDULED_ARRIVAL         0
ARRIVAL_TIME           4433
ARRIVAL_DELAY          4433
DIVERTED                  0
CANCELLED                 0
CANCELLATION_REASON       0
AIR_SYSTEM_DELAY       4433
SECURITY_DELAY         4433
AIRLINE_DELAY          4433
LATE_AIRCRAFT_DELAY    4433
WEATHER_DELAY          4433
dtype: int64

> **We notice that the null values differ between the 2 statuses. It is logically true that the features of the canceled flights differ from the features of the not canceled flights, So, The Nulls are normal here.**

> But we do not need the last five columns and we will stay with `CANCELLATION_REASON` instead of it.

In [415]:
needed_columns = df.columns[range(-5, 0, 1)]
df.drop(columns= needed_columns, inplace=True)
df.head(1)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON
0,2015,11,28,6,DL,1590,N971AT,CAE,ATL,700,655.0,-5.0,17.0,712.0,68.0,61.0,39.0,192,751.0,5.0,808,756.0,-12.0,0,0,NO-CANCELLATION


### Cleaning time columns
`SCHEDULED_DEPARTURE`, `DEPARTURE_TIME`, `SCHEDULED_TIME`, `ELAPSED_TIME`, `AIR_TIME`, `SCHEDULED_ARRIVAL`, `ARRIVAL_TIME` 

> We need to make them on the format **YYYY/MM/DD HH:MM** so:

> First we need to get the Date part for each row ... so we will concatinate the first 3 columns to get one column called `DATE`.

In [416]:
Date_Columns = df.columns[range(0,3, 1)] # getting the columns YEAR, MONTH, DAY

# Converting them to str to concatinating them with each other in one column called "DATE"
for column in Date_Columns:
    df[column] = df[column].astype(str)

Date_Format = df['YEAR'] + '/' + df['MONTH'] + '/' + df['DAY']
df.insert(0, "DATE", Date_Format)
df.DATE.head()

0    2015/11/28
1     2015/11/2
2     2015/8/22
3     2015/11/9
4     2015/6/12
Name: DATE, dtype: object

In [417]:
# Now we do not need the columns YEAR, MONTEH, DAY
df.drop(columns=Date_Columns, inplace=True)

> **Now we will format at first the time columns to be 'HH:MM'**

In [418]:
# Getting the needed columns names in one place.
time_columns = ['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'WHEELS_OFF', 'WHEELS_ON']

In [419]:
def adding_zeros(row_value):
    '''
    Input: the row value come from the getting_time_form function.
    Output: the new format after adding the needed number of zeos
    
    '''
    nedded_number_of_zeors = 4 - len(row_value)
    return  '0' *  nedded_number_of_zeors + row_value

def getting_time_form(row_val):
    '''
    
    Input: the current value in the column form convert_to_time function
    Output: returning it on the format HH : MM, but still string
    
    '''
    value = adding_zeros(row_val)
    hours = value[0:2] 
    minutes = value[2:4] 
    
    # If there is no problms with hours and minutes .. return this time.
    if (int(hours) >= 0 and int(hours) < 24) and (int(minutes) >= 0 and int(minutes) < 60):
        return hours + ':' + minutes
    
    # If there is a problem return Null. Like large hours than 23 or minutes than 59 or Nan value.
    else:
        # we can handel this error by taking the difference between the actual minutes - 60 to be the new minute.
        # Also we can handel hours like that.
        # But this will make many values unConsistant with each others as these times dependent on each others.
        
        # So, I will pass these values giving them Null as value.
        pass

In [420]:
for column in time_columns:
    df[column] = df[column].fillna(5555) # Convert nan/null to 5555 to consider it as a wrong time.
    df[column] = df[column].astype('int') # to get rid of the decimal sign.
    df[column] = df[column].astype('str') # to handel its length and add easily zeros.
    
    # Converting the value from the shape '####' to the shape 'HH:MM'
    df[column] = df[column].apply(lambda val: getting_time_form(val))
    
df[time_columns].head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,SCHEDULED_ARRIVAL,ARRIVAL_TIME,WHEELS_OFF,WHEELS_ON
0,07:00,06:55,None,None,00:39,08:08,07:56,07:12,07:51
1,17:15,17:43,None,None,01:26,18:59,19:40,18:23,19:29
2,13:55,13:53,02:20,02:06,None,16:35,16:19,14:13,16:15
3,06:50,06:52,None,None,01:42,08:51,08:44,07:08,08:30
4,07:05,07:00,None,None,None,08:38,08:32,07:12,08:23


> Now our time columns ready to be compied in the `DATE` column to get the format _YYYY/MM/DD HH:MM_

In [421]:
for column in time_columns:
    df[column] = df['DATE'] +' '+ df[column]
    df[column] = pd.to_datetime(df[column], format="%Y/%m/%d %H:%M")

    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274964 entries, 0 to 274963
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   DATE                 274964 non-null  object        
 1   DAY_OF_WEEK          274964 non-null  int64         
 2   AIRLINE              274964 non-null  object        
 3   FLIGHT_NUMBER        274964 non-null  int64         
 4   TAIL_NUMBER          274225 non-null  object        
 5   ORIGIN_AIRPORT       274964 non-null  object        
 6   DESTINATION_AIRPORT  274964 non-null  object        
 7   SCHEDULED_DEPARTURE  274964 non-null  datetime64[ns]
 8   DEPARTURE_TIME       270691 non-null  datetime64[ns]
 9   DEPARTURE_DELAY      270719 non-null  float64       
 10  TAXI_OUT             270571 non-null  float64       
 11  WHEELS_OFF           270538 non-null  datetime64[ns]
 12  SCHEDULED_TIME       143600 non-null  datetime64[ns]
 13  ELAPSED_TIME  

> Now we converted the time columns to Datetime columns ... Now lets convert the `DAY_OF_WEEK` column to be string with the day name.

In [422]:
df['DAY_OF_WEEK'] = pd.to_datetime(df.DATE).dt.day_name()
df.DAY_OF_WEEK.head()

0    Saturday
1      Monday
2    Saturday
3      Monday
4      Friday
Name: DAY_OF_WEEK, dtype: object

# Now, lets benifit from the airports dataset.
> **Notice, we will not run it ... as in TABLEAU it make problems with longtudes and latitudes.**

In [423]:
df.head(1)

,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON
0,2015/11/28,Saturday,DL,1590,N971AT,CAE,ATL,2015-11-28 07:00:00,2015-11-28 06:55:00,-5.0,17.0,2015-11-28 07:12:00,NaT,NaT,2015-11-28 00:39:00,192,2015-11-28 07:51:00,5.0,2015-11-28 08:08:00,2015-11-28 07:56:00,-12.0,0,0,NO-CANCELLATION


In [424]:
df_airports.head(3)

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919


In [425]:
# Lets first compine the Origin airport informations.

df = df.merge(df_airports, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE')
df.drop(columns=['IATA_CODE'], inplace=True)
df.head(1)

,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015/11/28,Saturday,DL,1590,N971AT,CAE,ATL,2015-11-28 07:00:00,2015-11-28 06:55:00,-5.0,17.0,2015-11-28 07:12:00,NaT,NaT,2015-11-28 00:39:00,192,2015-11-28 07:51:00,5.0,2015-11-28 08:08:00,2015-11-28 07:56:00,-12.0,0,0,NO-CANCELLATION,Columbia Metropolitan Airport,Columbia,SC,USA,33.93884,-81.11954


In [426]:
def Renaming_Columns(type_of_airport):
    '''
        INPUT: getting a string to add it to the columns of the merged airports
        OUTPUT: NONE - as it compine the data frame inplace.
    '''
    Columns_Names = df.columns[range(-6, 0, 1)]
    for column in Columns_Names:
        df.rename(columns={column: column + type_of_airport}, inplace=True)

In [427]:
Renaming_Columns("_ORIGIN")
df.head(2)

,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIRPORT_ORIGIN,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN
0,2015/11/28,Saturday,DL,1590,N971AT,CAE,ATL,2015-11-28 07:00:00,2015-11-28 06:55:00,-5.0,17.0,2015-11-28 07:12:00,NaT,NaT,2015-11-28 00:39:00,192,2015-11-28 07:51:00,5.0,2015-11-28 08:08:00,2015-11-28 07:56:00,-12.0,0,0,NO-CANCELLATION,Columbia Metropolitan Airport,Columbia,SC,USA,33.93884,-81.11954
1,2015/5/27,Wednesday,EV,5543,N846AS,CAE,ATL,2015-05-27 05:45:00,2015-05-27 05:40:00,-5.0,11.0,2015-05-27 05:51:00,NaT,2015-05-27 00:59:00,2015-05-27 00:43:00,192,2015-05-27 06:34:00,5.0,2015-05-27 06:45:00,2015-05-27 06:39:00,-6.0,0,0,NO-CANCELLATION,Columbia Metropolitan Airport,Columbia,SC,USA,33.93884,-81.11954


> Now, we can merge the DESTINATION_AIRPORT data.

In [428]:
# Lets compine the DESTINATION_AIRPORT informations.

df = df.merge(df_airports, left_on='DESTINATION_AIRPORT', right_on='IATA_CODE')
df.drop(columns=['IATA_CODE'], inplace=True)
df.head(1)

,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIRPORT_ORIGIN,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015/11/28,Saturday,DL,1590,N971AT,CAE,ATL,2015-11-28 07:00:00,2015-11-28 06:55:00,-5.0,17.0,2015-11-28 07:12:00,NaT,NaT,2015-11-28 00:39:00,192,2015-11-28 07:51:00,5.0,2015-11-28 08:08:00,2015-11-28 07:56:00,-12.0,0,0,NO-CANCELLATION,Columbia Metropolitan Airport,Columbia,SC,USA,33.93884,-81.11954,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694


In [429]:
Renaming_Columns("_DESTINATION")
df.head(2)

,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIRPORT_ORIGIN,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,AIRPORT_DESTINATION,CITY_DESTINATION,STATE_DESTINATION,COUNTRY_DESTINATION,LATITUDE_DESTINATION,LONGITUDE_DESTINATION
0,2015/11/28,Saturday,DL,1590,N971AT,CAE,ATL,2015-11-28 07:00:00,2015-11-28 06:55:00,-5.0,17.0,2015-11-28 07:12:00,NaT,NaT,2015-11-28 00:39:00,192,2015-11-28 07:51:00,5.0,2015-11-28 08:08:00,2015-11-28 07:56:00,-12.0,0,0,NO-CANCELLATION,Columbia Metropolitan Airport,Columbia,SC,USA,33.93884,-81.11954,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
1,2015/5/27,Wednesday,EV,5543,N846AS,CAE,ATL,2015-05-27 05:45:00,2015-05-27 05:40:00,-5.0,11.0,2015-05-27 05:51:00,NaT,2015-05-27 00:59:00,2015-05-27 00:43:00,192,2015-05-27 06:34:00,5.0,2015-05-27 06:45:00,2015-05-27 06:39:00,-6.0,0,0,NO-CANCELLATION,Columbia Metropolitan Airport,Columbia,SC,USA,33.93884,-81.11954,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694


> There is no need for the `DESTINATION_AIRPORT` and `ORIGIN_AIRPORT`.

In [430]:
df.drop(columns=['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'], inplace=True)

# Now, lets benifit from the airlines dataset.

In [431]:
df_airlines.head(2)

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.


In [432]:
df = df.merge(df_airlines, left_on='AIRLINE', right_on='IATA_CODE', suffixes=('_x', ''))
df.drop(columns=['IATA_CODE', 'AIRLINE_x'], inplace=True)
df.head(2)

,DATE,DAY_OF_WEEK,FLIGHT_NUMBER,TAIL_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIRPORT_ORIGIN,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,AIRPORT_DESTINATION,CITY_DESTINATION,STATE_DESTINATION,COUNTRY_DESTINATION,LATITUDE_DESTINATION,LONGITUDE_DESTINATION,AIRLINE
0,2015/11/28,Saturday,1590,N971AT,2015-11-28 07:00:00,2015-11-28 06:55:00,-5.0,17.0,2015-11-28 07:12:00,NaT,NaT,2015-11-28 00:39:00,192,2015-11-28 07:51:00,5.0,2015-11-28 08:08:00,2015-11-28 07:56:00,-12.0,0,0,NO-CANCELLATION,Columbia Metropolitan Airport,Columbia,SC,USA,33.93884,-81.11954,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Delta Air Lines Inc.
1,2015/4/28,Tuesday,1435,N909DE,2015-04-28 07:00:00,2015-04-28 06:55:00,-5.0,13.0,2015-04-28 07:08:00,NaT,2015-04-28 00:58:00,2015-04-28 00:40:00,192,2015-04-28 07:48:00,5.0,2015-04-28 08:08:00,2015-04-28 07:53:00,-15.0,0,0,NO-CANCELLATION,Columbia Metropolitan Airport,Columbia,SC,USA,33.93884,-81.11954,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Delta Air Lines Inc.


> ### Now, we can say that our work in cleaning the data is done. Now we will go to TABLEAU to analyze the data 

> **You can find the repo [here](https://github.com/mo7amed7assan1911/Flight_Delays_USA_TABLEAU)**

In [433]:
df.to_csv('flights_One_DataSet.csv')